# Notebook for 05_train_ms_from_scratch.py

###  Import libaries

In [8]:
import os
from glob import glob
from keras.applications.vgg16 import VGG16 as VGG
from keras.applications.densenet import DenseNet201 as DenseNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from image_functions import simple_image_generator

<div class="alert alert-block alert-info">Change path to datasets and model here:<div/>

### define path to training and validation data

In [9]:
# variables
path_to_split_datasets = "~/Documents/Data/PyCon/AllBands"
use_vgg = False
batch_size = 64

# contruct path
path_to_home = os.path.expanduser("~")
path_to_split_datasets = path_to_split_datasets.replace("~", path_to_home)
path_to_train = os.path.join(path_to_split_datasets, "train")
path_to_validation = os.path.join(path_to_split_datasets, "validation")

![tree](images_for_notebook/tree_files.png "file_tree")

### define classes

In [10]:
class_indices = {'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2,
                 'Highway': 3, 'Industrial': 4, 'Pasture': 5,
                 'PermanentCrop': 6, 'Residential': 7, 'River': 8,
                 'SeaLake': 9}
num_classes = len(class_indices)

## Training from scratch

![vgg16](images_for_notebook/vgg16.png "Original VGG")


### Initialize network model without top layers

![vgg16_no_top](images_for_notebook/vgg16_no_top.png "VGG no top")

In [11]:
# parameters for CNN
if use_vgg:
    base_model = VGG(include_top=False,
                     weights=None,
                     input_shape=(64, 64, 13))
else:
    base_model = DenseNet(include_top=False,
                          weights=None,
                          input_shape=(64, 64, 13))

### define new top layers

![vgg16_sentinel_rgb](images_for_notebook/vgg16_sentinel_rgb.png "VGG RGB Sentinel")

In [ ]:
# add a global spatial average pooling layer
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
# or just flatten the layers
#    top_model = Flatten()(top_model)
# let's add a fully-connected layer
if use_vgg:
    # only in VGG19 a fully connected nn is added for classfication
    # DenseNet tends to overfitting if using additionally dense layers
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dense(2048, activation='relu')(top_model)
# and a logistic layer
predictions = Dense(num_classes, activation='softmax')(top_model)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
# print network structure
model.summary()

### define data augmentation

In [13]:
# defining ImageDataGenerators
# ... initialization for training
training_files = glob(path_to_train + "/**/*.tif")
train_generator = simple_image_generator(training_files, class_indices,
                                         batch_size=batch_size,
                                         rotation_range=45,
                                         horizontal_flip=True,
                                         vertical_flip=True)

# ... initialization for validation
validation_files = glob(path_to_validation + "/**/*.tif")
validation_generator = simple_image_generator(validation_files, class_indices,
                                              batch_size=batch_size)

### define callbacks

In [14]:
# generate callback to save best model w.r.t val_categorical_accuracy
if use_vgg:
    file_name = "vgg"
else:
    file_name = "dense"
checkpointer = ModelCheckpoint("../data/models/" + file_name +
                               "_ms_from_scratch." +
                               "{epoch:02d}-{val_categorical_accuracy:.3f}." +
                               "hdf5",
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True,
                               mode='max')
earlystopper = EarlyStopping(monitor='val_categorical_accuracy',
                             patience=50,
                             mode='max',
                             restore_best_weights=True)

tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_grads=True,
                          write_images=True, update_freq='epoch')

![tensorflow](images_for_notebook/tensorflow.png "VGG RGB Sentinel")

### fit model

In [15]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=5,
        callbacks=[checkpointer, earlystopper, tensorboard],
        validation_data=validation_generator,
        validation_steps=500)

Epoch 1/5
100/100 [==============================] - 85s 847ms/step - loss: 1.0472 - categorical_accuracy: 0.6573 - val_loss: 1.4713 - val_categorical_accuracy: 0.6851

Epoch 00001: val_categorical_accuracy improved from -inf to 0.68509, saving model to ../data/models/dense_ms_from_scratch.01-0.685.hdf5
Epoch 2/5
100/100 [==============================] - 49s 491ms/step - loss: 0.6708 - categorical_accuracy: 0.7800 - val_loss: 0.7417 - val_categorical_accuracy: 0.7864

Epoch 00002: val_categorical_accuracy improved from 0.68509 to 0.78644, saving model to ../data/models/dense_ms_from_scratch.02-0.786.hdf5
Epoch 3/5
100/100 [==============================] - 48s 484ms/step - loss: 0.5662 - categorical_accuracy: 0.8178 - val_loss: 0.7565 - val_categorical_accuracy: 0.7695

Epoch 00003: val_categorical_accuracy did not improve from 0.78644
Epoch 4/5
100/100 [==============================] - 48s 477ms/step - loss: 0.5295 - categorical_accuracy: 0.8238 - val_loss: 0.4994 - val_categorical_